In [41]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

from numpy.linalg import norm, solve
import numpy as np

import pinocchio

In [42]:
data = np.loadtxt("optimal_solution.txt")

target_state = data[0].T
target_position = target_state[1:8]
target_velocity = target_state[8:15]

data_rk = data[1:202].T
time_rk = data_rk[0]
position_rk = data_rk[1:8]
velocity_rk = data_rk[8:15]
acceleration_rk = data_rk[15:22]
torque_rk = data_rk[22:29]

data_mpc = data[202:].T
time_mpc = data_mpc[0]
position_mpc = data_mpc[1:8]
velocity_mpc = data_mpc[8:15]
acceleration_mpc = data_mpc[15:22]
torque_mpc = data_mpc[22:29]


position_limit = np.array([[-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973], 
                          [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]])

velocity_limit = np.array([2.1750, 2.1750, 2.1750, 2.1750, 2.6100, 2.6100, 2.6100])

acceleration_limit = np.array([15.0, 7.5, 10.0, 12.5, 15.0, 20.0, 20.0])

torque_limit = np.array([87, 87, 87, 87, 12, 12, 12])

colors = px.colors.qualitative.Plotly
fig = make_subplots(rows=7, cols=4, column_titles=["position", "velocity", "acceleration", "torque"], row_titles=["Joint 1", "Joint 2", "Joint 3", "Joint 4", "Joint 5", "Joint 6", "Joint 7"])

for i in range(7):

    # Plot position trajectory, limit and target
    fig.add_trace(
    go.Scatter(x=time_mpc, y=position_mpc[i, :], line_color=colors[2]),
    row=i+1, col=1 )

    fig.add_trace(
    go.Scatter(x=time_rk, y=position_rk[i, :], opacity=0.3, line_color=colors[2]),
    row=i+1, col=1 )

    fig.add_hline(y=target_position[i],
    row=i+1, col=1, line_dash="dash", line_width=0.5, annotation_text="Target", annotation_font_size=8)

    fig.add_hline(y=position_limit[0, i],
    row=i+1, col=1, line_dash="dash", line_width=1)

    fig.add_hline(y=position_limit[1, i],
    row=i+1, col=1, line_dash="dash", line_width=1)

    # Plot velocitiy trajectory, limit and target
    fig.add_trace(
    go.Scatter(x=time_mpc, y=velocity_mpc[i, :], line_color=colors[9]),
    row=i+1, col=2 )

    fig.add_trace(
    go.Scatter(x=time_rk, y=velocity_rk[i, :], opacity=0.5, line_color=colors[9]),
    row=i+1, col=2 )

    fig.add_hline(y=target_velocity[i],
    row=i+1, col=2, line_dash="dash", line_width=0.5)

    fig.add_hline(y=velocity_limit[i],
    row=i+1, col=2, line_dash="dash", line_width=1)

    fig.add_hline(y=-velocity_limit[i],
    row=i+1, col=2, line_dash="dash", line_width=1)

    # Plot acceleration trajectory and limit
    fig.add_trace(
    go.Scatter(x=time_mpc, y=acceleration_mpc[i, :], line_color=colors[1]),
    row=i+1, col=3 )

    fig.add_trace(
    go.Scatter(x=time_rk, y=acceleration_rk[i, :], opacity=0.4, line_color=colors[1]),
    row=i+1, col=3 )

    fig.add_hline(y=acceleration_limit[i],
    row=i+1, col=3, line_dash="dash", line_width=1)

    fig.add_hline(y=-acceleration_limit[i],
    row=i+1, col=3, line_dash="dash", line_width=1)

    # Plot torque , limit and target
    fig.add_trace(
    go.Scatter(x=time_mpc, y=torque_mpc[i, :], line_color=colors[0]),
    row=i+1, col=4 )

    fig.add_trace(
    go.Scatter(x=time_rk, y=torque_rk[i, :], opacity=0.3, line_color=colors[0]),
    row=i+1, col=4 )

    fig.add_hline(y=torque_limit[i],
    row=i+1, col=4, line_dash="dash", line_width=1)

    fig.add_hline(y=-torque_limit[i],
    row=i+1, col=4, line_dash="dash", line_width=1)
title = "1SQP_700QP_10accel_90_pos"
fig.update_layout(height=2000, width=1000, title_text=title)
fig.update(layout_showlegend=False)
fig.show()

In [43]:
# fig.write_image("results_images/test_high_velocity/" + title + ".png")

In [44]:
model_pin = pinocchio.buildModelFromUrdf("../robot_utils/panda-model/panda_arm.urdf")
data_pin  = model_pin.createData()

task_position_mpc = np.zeros((3, (position_mpc.shape)[1]))

frameId = model_pin.getFrameId("panda_link8")

for i in range((position_mpc.shape)[1]):
    pinocchio.forwardKinematics(model_pin, data_pin, position_mpc[:, i])
    pinocchio.updateFramePlacement(model_pin, data_pin, frameId)
    task_position_mpc[:, i] = data_pin.oMf[frameId].translation

fig = make_subplots(rows=1, cols=3, column_titles=["X", "Y", "Z"])

for i in range(3):
    fig.add_trace(
    go.Scatter(x=time_mpc, y=task_position_mpc[i, :], opacity=0.3, line_color=colors[0]),
    row=1, col=i+1 )

title = "min02_maxInf"
fig.update_layout(height=400, width=1000, title_text=title)
fig.update(layout_showlegend=False)
fig.show()
fig.write_image("results_images/table_avoidance/" + title + ".png")

In [45]:
data.shape

(403, 29)